# [ Chapter 4 - Crowdsourced Relevance ] 
# Setting up the Retrotech Dataset

In [1]:
import sys
sys.path.append('..')
from aips import *  
from pyspark.sql import SparkSession
engine = get_engine()
spark = SparkSession.builder.appName("AIPS").getOrCreate()

## Download the Retrotech (Ecommerce) Products + Signals Dataset

In [5]:
#Get datasets
![ ! -d 'retrotech' ] && git clone --depth 1 https://github.com/ai-powered-search/retrotech.git
! cd retrotech && git pull
! cd retrotech && mkdir -p '../../data/reviews/' && tar -xvf products.tgz -C '../../data/retrotech/' && tar -xvf signals.tgz -C '../../data/retrotech/'

Already up to date.
products.csv
signals.csv


## Get a Feel for the Product Catalog

### Listing 4.1

In [6]:
! cd ../data/retrotech/ && head products.csv

"upc","name","manufacturer","shortDescription","longDescription"
"096009010836","Fists of Bruce Lee - Dolby - DVD",\N,\N,\N
"043396061965","The Professional - Widescreen Uncut - DVD",\N,\N,\N
"085391862024","Pokemon the Movie: 2000 - DVD",\N,\N,\N
"067003016025","Summerbreeze - CD","Nettwerk",\N,\N
"731454813822","Back for the First Time [PA] - CD","Def Jam South",\N,\N
"024543008200","Big Momma's House - Widescreen - DVD",\N,\N,\N
"031398751823","Kids - DVD",\N,\N,\N
"037628413929","20 Grandes Exitos - CD","Sony Discos Inc.",\N,\N
"060768972223","Power Of Trinity (Box) - CD","Sanctuary Records",\N,\N


## Index the Products into the Search Engine

### Listing 4.2

In [7]:
products_collection = engine.create_collection("products")
products_collection.write_from_csv("../data/retrotech/products.csv")

Wiping "products" collection
Creating "products" collection
Status: Success
Loading products
products Schema: 
root
 |-- upc: long (nullable = true)
 |-- name: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- shortDescription: string (nullable = true)
 |-- longDescription: string (nullable = true)

Status: Success


## Verify Searches Work

### Listing 4.3

In [8]:
# %load -s product_search_request solr
def product_search_request(query):
    return {
        "query": query,
        "fields": ["upc", "name", "manufacturer", "score"],
        "limit": 5,
        "params": {
            "qf": "name manufacturer longDescription",
            "defType": "edismax",
            "indent": "true",
            "sort": "score desc, upc asc"
        }
    }


In [9]:
query = "ipod"
request = product_search_request(query)
response = products_collection.search(request)
display_product_search(query, engine.docs_from_response(response))

{'upc': '878816004532', 'name': 'Fusion - Apple&#xAE; iPod&#xAE; Dock for Most Fusion 600 Series Stereos', 'manufacturer': 'Fusion', 'score': 2.7225723}
Fusion - Apple&#xAE; iPod&#xAE; Dock for Most Fusion 600 Series Stereos
{'upc': '694318011294', 'name': 'Alesis - Multimix 8-Channel USB 2.0 Mixer', 'manufacturer': 'Alesis', 'score': 2.7012062}
Alesis - Multimix 8-Channel USB 2.0 Mixer
{'upc': '885909300549', 'name': 'Apple - USB Charge/Sync Cable for Apple&#xAE; iPod&#xAE; shuffle', 'manufacturer': 'Apple', 'score': 2.6864247}
Apple - USB Charge/Sync Cable for Apple&#xAE; iPod&#xAE; shuffle
{'upc': '27108936499', 'name': 'Yamaha - Apple&#xAE; iPod&#xAE; and iPhone&#xAE; Dock for Most Yamaha A/V Receivers', 'manufacturer': 'Yamaha', 'score': 2.523007}
Yamaha - Apple&#xAE; iPod&#xAE; and iPhone&#xAE; Dock for Most Yamaha A/V Receivers
{'upc': '27242752436', 'name': 'Sony - 10W Micro Hi-Fi Shelf System with Built-in Apple&#xAE; iPod&#xAE; Dock', 'manufacturer': 'Sony', 'score': 2.523007

## Get a Feel for the Signals Data

In [ ]:
! cd ../data/retrotech && head signals.csv

"query_id","user","type","target","signal_time"
"u2_0_1","u2","query","nook","2019-07-31 08:49:07.3116"
"u2_1_2","u2","query","rca","2020-05-04 08:28:21.1848"
"u3_0_1","u3","query","macbook","2019-12-22 00:07:07.0152"
"u4_0_1","u4","query","Tv antenna","2019-08-22 23:45:54.1030"
"u5_0_1","u5","query","AC power cord","2019-10-20 08:27:00.1600"
"u6_0_1","u6","query","Watch The Throne","2019-09-18 11:59:53.7470"
"u7_0_1","u7","query","Camcorder","2020-02-25 13:02:29.3089"
"u9_0_1","u9","query","wireless headphones","2020-04-26 04:26:09.7198"
"u10_0_1","u10","query","Xbox","2019-09-13 16:26:12.0132"


## Index the Signals into the Search Engine

### Listing 4.4

In [ ]:
signals_collection = engine.create_collection("signals")
signals_collection.write_from_csv("../data/retrotech/signals.csv")

Wiping "signals" collection
Creating "signals" collection
Status: Success
Loading signals
signals Schema: 
root
 |-- query_id: string (nullable = true)
 |-- user: string (nullable = true)
 |-- type: string (nullable = true)
 |-- target: string (nullable = true)
 |-- signal_time: timestamp (nullable = true)

Status: Success


## Success!

You have now indexed the RetroTech product catalog and signals into the search engine, and run a sample query against the product collection. The results don't look very relevant using the out of the box keyword scoring function, of course, but we'll be working to improve that throughout the rest of this book!

In the next section, we'll take a look at our first crowd-sourced AI-powered search technique: [Signals Boosting](2.signals-boosting.ipynb). 